In [1]:
from collections import namedtuple
from typing import Iterable
import random

DiceFace = namedtuple("DiceFace", ["gear", "motion"])

class Dice():
    def __init__(self, gears: Iterable[int], motions: Iterable[int]):
        if not len(gears) == len(motions):
            raise UserWarning("gears and motion must have the same length") 
        self.faces = [DiceFace(gear, motion) for (gear, motion) in zip(gears, motions)]

    def roll(self) -> DiceFace:
        return random.choice(self.faces)

    def gears(self) -> Iterable[int]:
        return [face.gear for face in self.faces]

    def motions(self) -> Iterable[int]:
        return [face.motion for face in self.faces]      

class GhostCar():
    def __init__(self, white_die, black_die):
        self.white_die = white_die
        self.black_die = black_die

    def race(self, turns=10):
        return [self.black_die.roll() if turn % 2 == 0 else self.white_die.roll()
                 for turn in range(turns)]
